This notebook is set up to test code as needed.

In [ ]:
# import packages
import pandas as pd
import os

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930

year = 2020
path_prefix = f"{year}/"

In [ ]:
eia923_allocated = pd.read_csv(f'{outputs_folder()}{path_prefix}/eia923_allocated_{year}.csv', dtype=get_dtypes(), parse_dates=['report_date'])
plant_attributes = pd.read_csv(f"{outputs_folder()}{path_prefix}/plant_static_attributes_{year}.csv")
hourly_profiles = pd.read_csv(f'{outputs_folder()}{path_prefix}/hourly_profiles_{year}.csv', dtype=get_dtypes(), parse_dates=['report_date'])


#cems = pd.read_csv(f'{outputs_folder()}{path_prefix}/cems_{year}.csv', dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])
#partial_cems_scaled = pd.read_csv(f'{outputs_folder()}{path_prefix}/partial_cems_scaled_{year}.csv', dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])

In [ ]:
print("11. Exporting monthly and annual plant-level results")
# create a separate dataframe containing only the EIA data that is missing from cems
monthly_eia_data_to_shape = eia923_allocated[
    (eia923_allocated["hourly_data_source"] == "eia")
    & ~(eia923_allocated["fuel_consumed_mmbtu"].isna())
]

hourly_profiles = impute_hourly_profiles.convert_profile_to_percent(
    hourly_profiles,
    group_keys=["ba_code", "fuel_category", "profile_method"],
    columns_to_convert=["profile", "flat_profile"],
)

In [ ]:
plant_attributes

In [ ]:
monthly_eia_data_to_shape

In [ ]:
# aggregate data to plant level and add plant metadata
monthly_eia_data_to_shape = monthly_eia_data_to_shape.groupby(
    ["plant_id_eia", "report_date"], dropna=False
).sum().reset_index().merge(
    plant_attributes[["plant_id_eia", "fuel_category", "ba_code", "state"]]
)


In [ ]:
# for each ba, split data and shape
for ba in list(monthly_eia_data_to_shape.ba_code.unique()):
    # filter the data to the ba
    ba_data_to_shape = monthly_eia_data_to_shape[monthly_eia_data_to_shape["ba_code"] == ba].copy()

    shaped_eia_data = impute_hourly_profiles.shape_monthly_eia_data_as_hourly(
        ba_data_to_shape, hourly_profiles
    )

    combined_plant_data = data_cleaning.combine_plant_data(
        cems, partial_cems_subplant, partial_cems_plant, shaped_eia_data, "hourly"
    )